In [1]:
#%pip install pyspark==3.4.1

In [2]:
### Spark Setup
import os
# Set HADOOP_HOME to resolve native library issues on Windows
os.environ['HADOOP_HOME'] = r'C:\hadoop'

from pyspark.sql import SparkSession

In [3]:
### Prompts ###
# GEnerate the code to read the CSV file located at E:\data\orders_large_dataset into a Spark DataFrame with header and inferSchema options set to true. 
# Show the first few rows of the DataFrame.

In [3]:
os.environ['HADOOP_HOME'] = r'C:\hadoop'
os.environ['hadoop.home.dir'] = r'C:\hadoop'
os.environ['PATH'] += os.pathsep + r'C:\hadoop\bin'

# # 🔥 Disable native Hadoop IO (CRITICAL)
os.environ["HADOOP_OPTS"] = "-Dio.native.lib.available=false"

## Restart the kernel after setting the environment variables above ###

In [4]:
# Initialize SparkSession
spark = SparkSession.builder.appName("ReadCSV"). \
     config("spark.hadoop.io.native.lib.available", "false"). \
     config("spark.hadoop.fs.file.impl", "org.apache.hadoop.fs.LocalFileSystem"). \
     config("spark.hadoop.fs.raw.impl", "org.apache.hadoop.fs.RawLocalFileSystem").getOrCreate()
spark

In [6]:
print(spark.sparkContext._jsc.hadoopConfiguration().get("hadoop.version"))

None


In [5]:
# Read CSV file
df = spark.read.csv("E:\\data\\orders_large_dataset\\orders_1.csv", header=True)
df.show()

+--------------------+-----------+---------+--------------------+-----------------+----------------+
|            order_id|customer_id|seller_id|          order_date|order_total_price|  payment_method|
+--------------------+-----------+---------+--------------------+-----------------+----------------+
|1522bc72-c504-498...|     776403|   382525|2024-12-29T09:34:...|         37310.17|           Chase|
|215607d8-3ef6-438...|     589166|   999796|2024-11-16T13:22:...|          4439.84|            Visa|
|880e17fe-d3ba-41c...|      78263|   272330|2024-11-14T03:25:...|         33781.07|        Discover|
|f6d9af0f-85aa-4a3...|     682428|   666561|2024-06-28T20:55:...|         43879.03|           Chase|
|4ec0ac9c-aea4-478...|     378635|   146300|2024-08-09T23:11:...|           1210.1|           Chase|
|763d7536-24ba-458...|     280971|   831485|2024-08-26T21:59:...|          9059.23|           Chase|
|c3d26f4c-64a9-4c3...|     349189|   859336|2024-09-19T21:31:...|         38629.78|        

In [8]:
#df.count() #100,000,000

In [6]:
df1 = df.limit(10) # For testing purpose, limit to 1000 rows

In [7]:
## Write DataFrame to Parquet format
# Create the output directory if it doesn't exist
output_dir = r'E:\data_new\orders_large_dataset\parquet_output2'
os.makedirs(output_dir, exist_ok=True)

# Disable native libraries and set algorithm version to avoid Windows issues
# spark.conf.set("io.native.lib.available", "false")
# spark.conf.set("hadoop.io.native.lib.available", "false")
# spark.conf.set("mapreduce.fileoutputcommitter.algorithm.version", "2")

# Write with gzip compression to avoid native library dependency
#df1.write.format('parquet').mode('overwrite').option("compression", "gzip").save(output_dir)
df1.write.format('parquet').mode('overwrite').save(output_dir)

Py4JJavaError: An error occurred while calling o42.save.
: ExitCodeException exitCode=-1073741515: 
	at org.apache.hadoop.util.Shell.runCommand(Shell.java:1007)
	at org.apache.hadoop.util.Shell.run(Shell.java:900)
	at org.apache.hadoop.util.Shell$ShellCommandExecutor.execute(Shell.java:1212)
	at org.apache.hadoop.util.Shell.execCommand(Shell.java:1306)
	at org.apache.hadoop.util.Shell.execCommand(Shell.java:1288)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLo